# Text generation with LSTM

## The importance of the sampling strategy

生成文本时

1. 每次选概率最高的
2. 按概率选择下一个文本
3. softmax temperature：加入 temperature 参数，控制概率


## Implementing character-level LSTM text generation

In [3]:
import keras
import numpy as np


path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 [==============================] - 1s 2us/step
Corpus length: 600893


In [4]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


/tmp/ipykernel_2517/321955488.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_2517/321955488.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [5]:
from keras import layers


model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

2022-08-31 10:05:59.729451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-31 10:05:59.733715: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys


for epoch in range(1, 60):
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

epoch 1
1565/1565 [==============================] - 377s 239ms/step - loss: 1.9658
--- Generating with seed: "mean healthier. it is wisdom, worldly wisdom, to administer
"
------ temperature: 0.2
mean healthier. it is wisdom, worldly wisdom, to administer
and spirit of the man is the spirit and strengly and man has the self--and man and speal that the spirit of the spirit of the spirit of the spirit and the spirit of the spirit of the spirit of the spirit of the spirit of the man and special for the spirit and spirit of the man is the spirit of the special man have the self--and the spirit of the self--and in the far and spirit of the strengly and ------ temperature: 0.5
 of the self--and in the far and spirit of the strengly and probless and conscience of dispect of discition, the gentary to have that far the same to the most
hans it is feeloded things
to the epertion
for the seems of the more long--and itself the defires that the
spirits of man and the preadness of the command allis

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1565/1565 [==============================] - 327s 209ms/step - loss: 1.3550
--- Generating with seed: " fact that philosophizing concerning morals might be
conduct"
------ temperature: 0.2
 fact that philosophizing concerning morals might be
conduct the many of the refinement of the soul, and a man is a spirit of the super of the most self-conditions of the most self-condition of the most selfotogations of the most self-consequence of the best of the most selfotal and the fact of the soul, and the more and the selfots of the present of the experience of the more and such an and the present of the selfision of the most similar of the present ------ temperature: 0.5
present of the selfision of the most similar of the present in the pleasure. therefore, and concerning the development of our enthulith a something of the most distinguished and attained with the more necessity of the subject of the revilisonion and enthulithest are intellecting stronger period; it is a man of the soul, and t

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1565/1565 [==============================] - 351s 224ms/step - loss: 1.3311
--- Generating with seed: "oken, proud, incurable hearts (the cynicism of
hamlet--the c"
------ temperature: 0.2
oken, proud, incurable hearts (the cynicism of
hamlet--the conduct of the same all the soul of the present of the subjection of the subject of the conscience of the conscience of the science of the same the conscience of the most present of the sense of the sense of the sense of the sense of the conscience of the sense of the sense of the spoty of the sense and the world of the strength of the powerful of the same the fact of the sense of the sense of the ------ temperature: 0.5
erful of the same the fact of the sense of the sense of the very decession of the bose of wholly a sense, he are good still spread believe and sense, and more world of the belief in the preparation--in the senses of its allow in the only of the religious still belongs of the greater here as the greater in the great in the sin

...

i------ temperature: 1.0
 impulse of the science of the same profuming the attachs: it is errorwanted didness understands
and perpreaching, admoral conviding which lay
and
opinion, and finally yething honong ranbly
in an abond what the most prudence of go other. then well afford. has existing prevailed
foundations in the chamority is infliction only otherwise old repirt: "they wary, and it is the boddleness beforeest, what venture to losing, what cracbly pretimate parious greatest------ temperature: 1.2
t venture to losing, what cracbly pretimate parious greatest. it almost valuish acilly sticler -moinate interpolune instrubety of one's development psyet most new sancting etgaineds, of hitherto age, there may doneleners sensse itself hardiness wh hanfouron
it
so, the abot
know the imppression of repressed can be
strant, everything
a
mediocrisis which
knows always with
theorioy of youth nentiraf? veter-pitumnanity. toot odeal means. my causes
by great typeepoch 38
1565/1565 [===========

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1565/1565 [==============================] - 302s 193ms/step - loss: 1.2533
--- Generating with seed: "test possibilities, and how often
in the past the type man h"
------ temperature: 0.2
test possibilities, and how often
in the past the type man has always be all possibility of the contempt of the family and contempt of the sense of the subject of the superior of the fact of the sense of the subject of the contempt of the contention of the subjection of the subject of the superiority of the superior for the fact of the proper concern of man who has always also a superficial faith and for the subject of the sense of the most spirit is the s------ temperature: 0.5
and for the subject of the sense of the most spirit is the sense of will in the possibility of the free spirit is a contemption of men in the great existence of the world and super that the soul has not been the
historical soul was a soul is the regards to philosophy is the function is a sign as the general surpor of the scho

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



1565/1565 [==============================] - 314s 200ms/step - loss: 1.2484
--- Generating with seed: "dition. the individual is
almost automatically bound to rule"
------ temperature: 0.2
dition. the individual is
almost automatically bound to rule the same of the same the same the same of the same of the sense of the most surplus of the most fact of the subject of the most surplus of the same and for the same really and the subject of the same of the present and the most successful to the problem of the same the same will to the same and the most gratitude and desire the personal consequently the same of the same will to the same will the ------ temperature: 0.5
consequently the same of the same will to the same will the problem of the character in this devil of the spirit of the senses of consideration is the art of strain and true of the philosophers as a demands of morality in many one and the most something will understand them the for the consequences of the have a faculty of th

Note that by training a bigger model, longer, on more data, you can achieve gen-
erated samples that look much more coherent and realistic than this one. But, of
course, don’t expect to ever generate any meaningful text, other than by random
chance: all you’re doing is sampling data from a statistical model of which characters
come after which characters. Language is a communication channel, and there’s a
distinction between what communications are about and the statistical structure of
the messages in which communications are encoded.